# importeren van de Dataset
De data dat wordt gebruikt is gedownload van Kaggle, er is gekeken naar welke data potentieel veel over te vinden is.
Hier is een dataset over de prijzen van avocados gevonden dat goed kan worden gebruikt bij het maken van deze
opdrachten.

In de code hieronder importeren we  de pandas library en laden we de avocado dataset in op de variabelen 'df'.


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np
import matplotlib

In [2]:
data = pd.read_csv('data/avocado.csv')
data.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


## Multiple-Linear-Regression 
In dit hoofdstuk gaan we een voorbeeld van Multiplaire Lineaire Regressie (MLR) uitwerken en evalueren. Met MLR gaan we
proberen de prijzen van de avocados te voorspellen gegeven de waardes van andere kollomen.

formule voor MLR: <br/>
\begin{equation}
\ y_i = \beta_0 x_i{_0} + \beta_1 x_i{_1}... + \beta_p x_i{_p} + {m_i}
\end{equation}


### Wat gaan we analyseren
Met behulp van MLR gaan we de prijzen van de advocado's voorspellen, de factoren die we hier voor gaan gebruiken zijn:
1. totale volume
2. Total Bags
3. Aantal per soort bag: Small, Large, XLarge
4. PLU (Price Lookup Codes) waardes: 4046, 4225, 4770

Mogelijke behulpzame factoren:
1. Jaar
2. Type: organisch of conventioneel
3. Rgio

### Uitwerking
#### Schoonmkaen
Allereerst moet er worden gecontroleerd worden hoeveel null waardes in de dataset zijn, pandas heeft hier een simpel
maar elegant truckje voor:

In [3]:
data.isnull().sum()

Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

Zoals te zien is de data schoon, dit is maar een eerste indruk dus er kan nog altijd wat fouten inzitten. Een van deze
fouten kon al in het begin worden opgemerkt toen de .head() van de dataset werdt getoond; de kollommen zijn niet
geindexeerd en deze kollom wordt niet goed als datum object herkend.

Om dit op te lossen wordt het voglende gedaan:

In [4]:
data["Date"] = pd.to_datetime(data["Date"], yearfirst=True)
data.set_index("Date", inplace=True)
data.sort_index(inplace=True)

In [5]:
data.head(3)

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
Date,,,,,,,,,,,,,
2015-01-04,51,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,organic,2015,Southeast
2015-01-04,51,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,organic,2015,Chicago
2015-01-04,51,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,organic,2015,HarrisburgScranton


Door het opnieuw indelen van de index is de oude index, wat een gewoon geheel getal was, als een kollom toegevoegd,
deze is niet nodig en kan dus verwijderd worden

In [6]:
data.drop(["Unnamed: 0"], axis=1, inplace=True)


#### Factoren en coëfficiënten

Nu de dataset logisch gewijs klopt kunnen de factoren worden gedefineerd, allereerst gaan we enkele kollomen weghalen
die geen continu data bevatten:

1. Type
2. Jaar
3. Region

De kollomen zeggen uiteindelijk wel iets over de prijs, maar aangezien we MLR gebruiken kunnen we deze data niet
in het model meenemen. Voor bij het gebruik van de categorische data (Type en Region) kan Multinomale Logische Regressie
worden gebruikt, maar aangezien dat buiten de scope van het onderzoek valt, wordt deze niet gebruikt. Year is discrete
data en daar zal Possion Regressie voor worden gebruikt, maar valt net als de categorische data buiten de scope.

In [18]:
data_continue = data.drop(["type", "region", "year"], axis=1).dropna()
data_continue.head()

,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags
Date,,,,,,,,,
2015-01-04,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0
2015-01-04,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0
2015-01-04,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0
2015-01-04,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0
2015-01-04,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0


Ten tweede wordt vanuit deze cotinue data de factoren opgebouwd, hiervoor gaan we sci-kit learn gebruiken:

In [19]:
Xs = data_continue.drop(["AveragePrice"], axis=1).values
y = data_continue["AveragePrice"].values
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=0)

Als laatst gaan we de lineaire regressie opbouwen, hiervoor gebruiken we weer sci-kit-learn.

In [20]:
reg = LinearRegression(normalize=True)
reg.fit(X_train, y_train)
print(reg.intercept_) 
print(reg.coef_)

1.4205921802106742
[ 0.00020275 -0.00020286 -0.00020269 -0.0002032  -0.02537155  0.02516883
  0.0251687   0.02517015]


#### Voorspellen van prijzen
Nu de coëfficiënten zijn bepaald voor de factoren kunnen we dit model gebruiken om voorspellingen op de prijzen te maken

In [21]:
y_voorspelling = reg.predict(X_test)
voorspellingen_df = pd.DataFrame({'Werkelijk': y_test, 'Voorspelt': y_voorspelling})
voorspellingen_df.head(20)

,Werkelijk,Voorspelt
0,1.11,1.414800
1,1.68,1.421365
2,2.11,1.421134
3,2.15,1.391286
4,1.70,1.424736
5,1.91,1.420939
6,0.76,1.370642
7,1.93,1.423610
8,1.42,1.418669
9,1.20,1.419781


### Evaluatie
om te evalueren hoe goed de voorspellingen bestaan enkele rekenmethodes om de error te berekenen, iedere rekenmethode is
een manier om de error uit te rekenen en geven ieder een ander soort waarde aan. Maar de waardes moeten eerst tegen iets
worden opgewogen; het algemene gemiddelde van de dataset.

In [32]:
y_gemiddeld = np.empty(y_test.shape, y_test.dtype)
y_gemiddeld.fill(np.mean(y_test))
print(y_gemiddeld)

[1.41353151 1.41353151 1.41353151 ... 1.41353151 1.41353151 1.41353151]


#### Mean Absolute Error (MAE)
De MAE geeft aan hoeveel de absolute afwijking per voorspelling is, zo te zien is het MLR model gemiddeld 31 cent
afwijkend van de daadwerkelijke prijs. Vergeleken met de 32 cent van het gemiddelde, een kleine verbeterin, maar
nogsteeds slecht

Vergeleken met voorspelling:

In [34]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_voorspelling)) 

Mean Absolute Error: 0.3135968009826497


Vergeleken met het gemiddelde van y_test

In [35]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_gemiddeld)) 

Mean Absolute Error: 0.32494888647025705


#### Mean Squared Error (MSE) 
Met MSE geef je de kwadratische error aan, hiermee kan je testen of er veel outliers in data zitten, zoals te zien is
dat de error relatief in de buurt is vergeleken met de MAE zijn er niet veel outliers in de dataset.

In [36]:
print('Mean Squared Error:', mean_squared_error(y_test, y_voorspelling))

Mean Squared Error: 0.15359315535007795


Vergeleken met het gemiddelde van y_test

In [37]:
print('Mean Absolute Error:', mean_squared_error(y_test, y_gemiddeld)) 

Mean Absolute Error: 0.1632669531169075


#### Root Mean Squared Error (RMSE)
Gezien de MSE een klein getal ( >1 ) is de RMSE geen goede meetmethode voor de uitkomsten bij deze dataset.

In [38]:
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_voorspelling))) 

Root Mean Squared Error: 0.39190962650855865


Vergeleken met het gemiddelde van y_test

In [39]:
print('Mean Absolute Error:', np.sqrt(mean_squared_error(y_test, y_gemiddeld)))

Mean Absolute Error: 0.40406305586740726


### Conclusie
MLR werkt maar de eigenschappen van de dataset hebben weinig correlatie met elkaar, waardoor bij het voorspellen van de
gemiddelde prijs het iets beter gaat dan het gemiddelde gebruiken, maar het zo klein het verschil is dat het bijna
nutteloos is.

## K-nearest Neighbour
In dit hoofdstuk gaan we een voorbeeld van K-nearest Neighbour uitwerken en evalueren. Door middel van deze analyse
techniek wordt geprobeerd te voorspellen wat voor type de avocado is

### Wat gaan we analyseren

### Uitwerking

### Evaluatie


## Naives Bayes
In dit hoofdstuk gaan we een voorbeeld van Nayves Bayes uitwerken en evalueren

### Wat gaan we analyseren

### Uitwerking

### Evaluatie